### Imports

In [206]:
import pandas as pd
from pymongo import MongoClient
import os

### File Paths

In [207]:
file_best_cities = r"E:\Bicocca\Data Management\workation\best_significant_european_workation_cities.xls"
file_cost_of_living = r"E:\Bicocca\Data Management\cost_of_living\significant_european_cities_cost_of_living.xls"
file_traffic = r"E:\Bicocca\Data Management\traffic\significant_european_cities_traffic_index.xls"
file_health = r"E:\Bicocca\Data Management\health_care\significant_european_cities_health_index.xls"
file_crime_and_safety = r"E:\Bicocca\Data Management\crime_and_safety\significant_european_cities_crime_safety_index.xls"
file_pollution = r"E:\Bicocca\Data Management\pollution\significant_european_cities_pollution_index.xls"

### Load Data

In [208]:
# Read each Excel file into a DataFrame
df_best_cities = pd.read_csv(file_best_cities)
df_cost_of_living = pd.read_csv(file_cost_of_living)
df_traffic = pd.read_csv(file_traffic)
df_health = pd.read_csv(file_health)
df_crime_and_safety = pd.read_csv(file_crime_and_safety)
df_pollution = pd.read_csv(file_pollution)

### Clean Column Names

In [209]:
def clean_columns(df):
    """
    Strips spaces, converts to lowercase, and replaces spaces with underscores.
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

# Apply cleaning to all DataFrames
df_best_cities = clean_columns(df_best_cities)
df_cost_of_living = clean_columns(df_cost_of_living)
df_traffic = clean_columns(df_traffic)
df_health = clean_columns(df_health)
df_crime_and_safety = clean_columns(df_crime_and_safety)
df_pollution = clean_columns(df_pollution)

# Preview
print("--- Cleaned Column Names ---")
print("\nBest Cities:", df_best_cities.columns.tolist())
print("\nCost of Living:", df_cost_of_living.columns.tolist())
print("\nTraffic:", df_traffic.columns.tolist())
print("\nHealth Care:", df_health.columns.tolist())
print("\nCrime And Safety:", df_crime_and_safety.columns.tolist())
print("\nPollution:", df_pollution.columns.tolist())

--- Cleaned Column Names ---

Best Cities: ['rank', 'city', 'country', 'wifi_speed_mbps', 'coworking_count', 'coffee_price', 'taxi_price_km', 'one_bed_apt_price', 'meal_price', 'sunshine_hours', 'attractions_count', 'ig_photos_count', 'beer_price', 'city_index']

Cost of Living: ['city_index', 'rank', 'city', 'cost_of_living_index', 'rent_index', 'cost_of_living_plus_rent_index', 'groceries_index', 'restaurant_price_index', 'local_purchasing_power_index', 'data_year', 'country']

Traffic: ['city', 'country', 'traffic_index', 'time_index', 'data_source']

Health Care: ['city', 'country', 'health_care_index', 'data_source']

Crime And Safety: ['city', 'country', 'crime_index', 'safety_index', 'data_source']

Pollution: ['city', 'country', 'pollution_index', 'data_source']


### Drop Unnecessary Columns

In [210]:
# Columns to remove from all DataFrames
cols_to_delete = [
    'one_bed_apt_price',
    'ig_photos_count',
    'local_purchasing_power_index',
    'data_year',
    'time_exp_index',
    'inefficiency_index',
    'co2_emission_index',
    'crime_index',
    'rank',          
    'city_index'     
]

# Apply safely to each DataFrame
for df in [df_best_cities, df_cost_of_living, df_traffic, df_health]:
    df.drop(columns=[col for col in cols_to_delete if col in df.columns], inplace=True)

# Confirm cleanup
print("Columns after cleanup:")
for name, df in zip(
    ['Best Cities', 'Cost of Living', 'Traffic', 'Health Care'], 
    [df_best_cities, df_cost_of_living, df_traffic, df_health]
):
    print(f"{name}: {df.columns.tolist()}")


Columns after cleanup:
Best Cities: ['city', 'country', 'wifi_speed_mbps', 'coworking_count', 'coffee_price', 'taxi_price_km', 'meal_price', 'sunshine_hours', 'attractions_count', 'beer_price']
Cost of Living: ['city', 'cost_of_living_index', 'rent_index', 'cost_of_living_plus_rent_index', 'groceries_index', 'restaurant_price_index', 'country']
Traffic: ['city', 'country', 'traffic_index', 'time_index', 'data_source']
Health Care: ['city', 'country', 'health_care_index', 'data_source']


### Merge DataFrames

In [217]:
# --- Step 0: Drop 'country' & 'data_source' from all except df_best_cities ---
df_cost_of_living_clean = df_cost_of_living.drop(columns=['country', 'data_source'], errors='ignore')
df_traffic_clean = df_traffic.drop(columns=['country', 'data_source'], errors='ignore')
df_health_clean = df_health.drop(columns=['country', 'data_source'], errors='ignore')
df_crime_and_safety_clean = df_crime_and_safety.drop(columns=['country', 'data_source'], errors='ignore')
df_pollution_clean = df_pollution.drop(columns=['country', 'data_source'], errors='ignore')

# --- Step 1: Merge Best Cities & Cost of Living ---
merged_df = pd.merge(df_best_cities, df_cost_of_living_clean, on='city', how='outer')

# --- Step 2: Merge with Traffic ---
merged_df = pd.merge(merged_df, df_traffic_clean, on='city', how='outer')

# --- Step 3: Merge with Health Care ---
merged_df = pd.merge(merged_df, df_health_clean, on='city', how='outer')

# --- Step 4: Merge with Crime and Safety ---
merged_df = pd.merge(merged_df, df_crime_and_safety_clean, on='city', how='outer')

# --- Step 5: Merge with Pollution ---
merged_df = pd.merge(merged_df, df_pollution_clean, on='city', how='outer')

# --- Step 5a: Clean city/country duplicates & spelling issues
CITY_NAME_MAPPING = {
    "Hanover": "Hannover",
    "Kiev (Kyiv)": "Kyiv",
    "Kyiv": "Kyiv",
    "Krakow (Cracow)": "Krakow",
    "Saint Petersburg": "St. Petersburg",
    "St. Petersburg": "St. Petersburg",
    "Seville (Sevilla)": "Seville"
}
merged_df['city'] = merged_df['city'].apply(lambda x: CITY_NAME_MAPPING.get(x, x))

COUNTRY_MAPPING = {
    "Hannover": "Germany",
    "Kyiv": "Ukraine",
    "Krakow": "Poland",
    "St. Petersburg": "Russia",
    "Seville": "Spain"
}
merged_df['country'] = merged_df.apply(
    lambda row: COUNTRY_MAPPING.get(row['city'], row['country']),
    axis=1
)

# Consolidate duplicates (keep first non-null values per city)
merged_df = merged_df.groupby(['city', 'country'], as_index=False).first()

# --- Step 6: Reset index and reorder columns ---
merged_df = merged_df.sort_values(by='city').reset_index(drop=True)

primary_cols = [c for c in ['city', 'country', 'data_source'] if c in merged_df.columns]
other_cols = [c for c in merged_df.columns if c not in primary_cols]
merged_df = merged_df[primary_cols + other_cols]

# --- Preview ---
merged_df.head()

,city,country,wifi_speed_mbps,coworking_count,coffee_price,taxi_price_km,meal_price,sunshine_hours,attractions_count,beer_price,...,rent_index,cost_of_living_plus_rent_index,groceries_index,restaurant_price_index,traffic_index,time_index,health_care_index,crime_index,safety_index,pollution_index
0,Amsterdam,Netherlands,22.0,99.0,2.85,2.01,12.81,1662.0,1782.0,4.27,...,49.5,59.7,58.0,76.6,76.0,22.1,81.5,25.7,74.3,22.6
1,Athens,Greece,12.0,29.0,2.42,0.64,8.54,2773.0,1149.0,3.42,...,15.1,33.3,46.4,51.8,151.9,37.2,58.3,55.2,44.8,55.2
2,Barcelona,Spain,37.0,136.0,1.59,1.01,10.25,2591.0,2739.0,2.56,...,29.6,41.3,49.3,53.3,99.0,30.0,76.7,51.9,48.1,63.0
3,Belfast,United Kingdom,26.0,13.0,2.74,1.07,11.98,1868.0,483.0,4.50,...,24.7,41.8,54.0,68.4,152.1,37.6,70.7,48.0,52.0,26.5
4,Belgrade,Serbia,34.0,33.0,1.32,0.51,5.08,2112.0,535.0,1.45,...,17.2,29.7,36.5,41.2,137.9,36.5,53.1,37.8,62.2,69.2


### Normalize Selected Columns

In [218]:
# Function to normalize a pandas Series between 0 and 1
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Select all numeric columns automatically
numeric_cols = merged_df.select_dtypes(include=['number']).columns.tolist()
print("Numeric columns to normalize:", numeric_cols)

# Apply normalization
for col in numeric_cols:
    merged_df[col] = normalize(merged_df[col])

print("\n--- All numeric columns normalized successfully ---")
merged_df[numeric_cols].head()

Numeric columns to normalize: ['wifi_speed_mbps', 'coworking_count', 'coffee_price', 'taxi_price_km', 'meal_price', 'sunshine_hours', 'attractions_count', 'beer_price', 'cost_of_living_index', 'rent_index', 'cost_of_living_plus_rent_index', 'groceries_index', 'restaurant_price_index', 'traffic_index', 'time_index', 'health_care_index', 'crime_index', 'safety_index', 'pollution_index']

--- All numeric columns normalized successfully ---


,wifi_speed_mbps,coworking_count,coffee_price,taxi_price_km,meal_price,sunshine_hours,attractions_count,beer_price,cost_of_living_index,rent_index,cost_of_living_plus_rent_index,groceries_index,restaurant_price_index,traffic_index,time_index,health_care_index,crime_index,safety_index,pollution_index
0,0.235294,0.311321,0.543909,0.652632,0.587292,0.212303,0.346201,0.520958,0.537179,0.800407,0.624615,0.418227,0.628190,0.058264,0.153392,0.781914,0.157560,0.842440,0.153030
1,0.039216,0.091195,0.422096,0.171930,0.333729,0.726179,0.216912,0.393713,0.282051,0.099796,0.218462,0.273408,0.326853,0.509512,0.598820,0.202348,0.743692,0.256308,0.646970
2,0.529412,0.427673,0.186969,0.301754,0.435273,0.641998,0.541667,0.264970,0.317949,0.395112,0.341538,0.309613,0.345079,0.195006,0.386431,0.662003,0.678124,0.321876,0.765152
3,0.313725,0.040881,0.512748,0.322807,0.538005,0.307586,0.080882,0.555389,0.378205,0.295316,0.349231,0.368290,0.528554,0.510702,0.610619,0.512116,0.600636,0.399364,0.212121
4,0.470588,0.103774,0.110482,0.126316,0.128266,0.420444,0.091503,0.098802,0.176923,0.142566,0.163077,0.149813,0.198056,0.426278,0.578171,0.072446,0.397973,0.602027,0.859091


### Data Quality Check

In [219]:
def data_quality_check(merged_df):
    total_rows = len(merged_df)

    # Treat empty strings / spaces as NaN
    merged_df = merged_df.replace(r'^\s*$', np.nan, regex=True)

    garbage_values = ['n/a', 'na', 'null', 'none', 'unknown', '-', '--']
    summary = []

    # --- Country/City specific checks ---
    country_city_same_count = 0
    city_in_country_count = 0

    if {'country', 'city'}.issubset(merged_df.columns):
        # Normalize text
        country_series = merged_df['country'].str.lower().str.strip()
        city_series = merged_df['city'].str.lower().str.strip()

        # Rule 1: country == city
        country_city_same_count = (country_series == city_series).sum()

        # Rule 2: city value appears in country column
        country_values = set(country_series.dropna().unique())
        city_in_country_count = city_series.isin(country_values).sum()

    # --- Column-wise checks ---
    for col in merged_df.columns:
        null_count = merged_df[col].isna().sum()
        null_pct = round((null_count / total_rows) * 100, 2)

        garbage_count = 0
        negative_count = 0
        zero_count = 0

        if merged_df[col].dtype == 'object':
            garbage_count = merged_df[col].str.lower().isin(garbage_values).sum()

        if pd.api.types.is_numeric_dtype(merged_df[col]):
            negative_count = (merged_df[col] < 0).sum()
            zero_count = (merged_df[col] == 0).sum()

        summary.append({
            'column': col,
            'null_count': null_count,
            'null_percentage': null_pct,
            'garbage_values': int(garbage_count),
            'negative_values': int(negative_count),
            'zero_values': int(zero_count),
            'unique_values': merged_df[col].nunique()
        })

    summary_df = pd.DataFrame(summary)

    duplicate_rows = merged_df.duplicated().sum()

    # --- Rows with any null value ---
    rows_with_nulls = merged_df[merged_df.isna().any(axis=1)][['city', 'country'] + [c for c in merged_df.columns if c not in ['city','country']]]

    # 📊 Human-readable report
    print("\n📊 DATA QUALITY SUMMARY")
    print("-" * 40)
    print(f"Total rows                  : {total_rows}")
    print(f"Total columns               : {merged_df.shape[1]}")
    print(f"Duplicate rows              : {duplicate_rows}")
    print(f"Columns with nulls          : {(summary_df['null_count'] > 0).sum()}")
    print(f"Columns with garbage        : {(summary_df['garbage_values'] > 0).sum()}")
    print(f"Columns with negative       : {(summary_df['negative_values'] > 0).sum()}")
    print(f"Country == City rows        : {country_city_same_count}")
    print(f"City value found in Country : {city_in_country_count}")

    if (
        duplicate_rows > 0
        or summary_df[['null_count', 'garbage_values', 'negative_values']].any().any()
        or country_city_same_count > 0
        or city_in_country_count > 0
    ):
        print("\n❌ Data quality issues found")
    else:
        print("\n✅ Data looks clean and ready")

    if not rows_with_nulls.empty:
        print("\n--- Rows with Missing Values (city & country included) ---")
        print(rows_with_nulls)

    return summary_df, rows_with_nulls

# Run the check
data_quality_summary, rows_with_null_values = data_quality_check(merged_df)


📊 DATA QUALITY SUMMARY
----------------------------------------
Total rows                  : 64
Total columns               : 21
Duplicate rows              : 0
Columns with nulls          : 0
Columns with garbage        : 0
Columns with negative       : 0
Country == City rows        : 0
City value found in Country : 0

✅ Data looks clean and ready


### Preview 

In [220]:
merged_df.head()

,city,country,wifi_speed_mbps,coworking_count,coffee_price,taxi_price_km,meal_price,sunshine_hours,attractions_count,beer_price,...,rent_index,cost_of_living_plus_rent_index,groceries_index,restaurant_price_index,traffic_index,time_index,health_care_index,crime_index,safety_index,pollution_index
0,Amsterdam,Netherlands,0.235294,0.311321,0.543909,0.652632,0.587292,0.212303,0.346201,0.520958,...,0.800407,0.624615,0.418227,0.628190,0.058264,0.153392,0.781914,0.157560,0.842440,0.153030
1,Athens,Greece,0.039216,0.091195,0.422096,0.171930,0.333729,0.726179,0.216912,0.393713,...,0.099796,0.218462,0.273408,0.326853,0.509512,0.598820,0.202348,0.743692,0.256308,0.646970
2,Barcelona,Spain,0.529412,0.427673,0.186969,0.301754,0.435273,0.641998,0.541667,0.264970,...,0.395112,0.341538,0.309613,0.345079,0.195006,0.386431,0.662003,0.678124,0.321876,0.765152
3,Belfast,United Kingdom,0.313725,0.040881,0.512748,0.322807,0.538005,0.307586,0.080882,0.555389,...,0.295316,0.349231,0.368290,0.528554,0.510702,0.610619,0.512116,0.600636,0.399364,0.212121
4,Belgrade,Serbia,0.470588,0.103774,0.110482,0.126316,0.128266,0.420444,0.091503,0.098802,...,0.142566,0.163077,0.149813,0.198056,0.426278,0.578171,0.072446,0.397973,0.602027,0.859091


### Save to MongoDB

In [222]:
# ------------------------------
# SAVE MERGED DATA TO MONGODB
# ------------------------------
client = MongoClient("mongodb://localhost:27017/")
db = client["data_management_project"] 
collection = db["significant_european_cities_quality_of_life"]      

print("Connected to MongoDB successfully!")

# Convert DataFrame to dictionary records and insert
data_dict = merged_df.to_dict("records")
collection.insert_many(data_dict)

print("Data saved to MongoDB successfully!")

Connected to MongoDB successfully!
Data saved to MongoDB successfully!
